# **Imports** (must run)

In [ ]:
# import dependencies
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from IPython.display import display, Javascript, Image

from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

#Get Python and OpenCV Version

print('OpenCV-Python Lib Version:', cv2.__version__)
print('Python Version:',sys.version)

OpenCV-Python Lib Version: 4.6.0
Python Version: 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


# **Camera Image Capture** (must run)

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# **Object Detection Javascript** (must run)

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom()
    {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame()
    {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom()
    {
      if (div !== null)
      {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData)
    {
      if (shutdown)
      {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "")
      {
        labelElement.innerHTML = label;
      }

      if (imgData != "")
      {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject)
      {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

**# Photo Object Recognition**

In [ ]:

img = cv2.imread("face.jpg")

# OpenCV opens images as BRG
# but we want it as RGB and
# we also need a grayscale
# version
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

stop_data = cv2.CascadeClassifier('eye.xml')

found = stop_data.detectMultiScale(img_gray,
                                   minSize =(100, 100))

# Don't do anything if there's
# no sign
amount_found = len(found)

if amount_found != 0:

    # There may be more than one
    # sign in the image
    for (x, y, width, height) in found:

        # We draw a green rectangle around
        # every recognized sign
        cv2.rectangle(img_rgb, (x, y),
                      (x + height, y + width),
                      (0, 255, 0), 5)

# Creates the environment of
# the picture and shows it
plt.subplot(1, 1, 1)
plt.imshow(img_rgb)
plt.show()

error: ignored

In [ ]:

img = cv2.imread("Car.jpg")

# OpenCV opens images as BRG
# but we want it as RGB and
# we also need a grayscale
# version
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

stop_data = cv2.CascadeClassifier('CarSideViewXML.xml')

found = stop_data.detectMultiScale(img_gray,
                                   minSize =(100, 100))

# Don't do anything if there's
# no sign
amount_found = len(found)

if amount_found != 0:

    # There may be more than one
    # sign in the image
    for (x, y, width, height) in found:

        # We draw a green rectangle around
        # every recognized sign
        cv2.rectangle(img_rgb, (x, y),
                      (x + height, y + width),
                      (0, 255, 0), 5)

# Creates the environment of
# the picture and shows it
plt.subplot(1, 1, 1)
plt.imshow(img_rgb)
plt.show()

error: ignored

# **Necessary Functions** (must run)

In [ ]:
def detectColor(img, color):
  if(color == "blue"):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    BlueLower = np.array([235,95,35]) #darker color
    BlueUpper = np.array([225,75,20]) #lighter color
    img_mask = cv2.inRange(img_hsv, BlueLower, BlueUpper)
    masked_out = cv2.bitwise_and(img, img, mask = img_mask)
  elif(color == "black"):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    BlackLower = np.array([0,0,0]) #darker color
    BlackUpper = np.array([236,21,39]) #lighter color
    img_mask = cv2.inRange(img_hsv, BlackLower, BlackUpper)
    masked_out = cv2.bitwise_and(img, img, mask = img_mask)
  elif(color == "white"):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    WhiteLower = np.array([236,16,78]) #darker color
    WhiteUpper = np.array([0,0,100]) #lighter color
    img_mask = cv2.inRange(img_hsv, WhiteLower, WhiteUpper)
    masked_out = cv2.bitwise_and(img, img, mask = img_mask)
  return img

In [ ]:
def withinRange(n, nAvg, value):
  return ((nAvg - value) < n < (nAvg + value))

In [ ]:
def drawCarBoundingBox(carType, bbox_array, yCoordList, yInterval, boundaryBoxThickness):
    for (x,y,w,h) in carType: #x, y, width (max: 1280px), height (max: 720px)
      # get the average of the coordinate values
      yAvg = sum(yCoordList)
      if(len(yCoordList) > 0):
        yAvg /= len(yCoordList)
      # to prevent any non-vehicle areas to be mistaken as a vehicle
      if(len(yCoordList) <= 20) or (withinRange(y,yAvg,yInterval)): # the vehicle will only move on a x-axis basis, so only check y-axis change
        if(len(yCoordList) >= 20):
          yCoordList.pop(19) # remove last index of the coordList
        yCoordList.append(y) # add new coord to the list
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x + w, y + h),(132,245,66),boundaryBoxThickness)

# **Main Challenge1** (car boundary)

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
boundaryBoxThickness = 5
scaleValue = 1.2
minNeigh = 3
yInterval = 150

# initialize the Haar Cascade car detection model
car_cascade = cv2.CascadeClassifier("CarSideViewXML.xml")
# initate a coordList that keeps the memory of the 20 y coordinates
yCoordList =[]

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"]) # convert JS response to OpenCV Image
    bbox_array = np.zeros([480,640,4], dtype = np.uint8) # create transparent overlay for bounding box

    # grayscale for the respective car color for car detection
    blue = detectColor(img, "blue") #cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    black = detectColor(img, "black")
    white = detectColor(img, "white")

    # get the region coordinates respective car color
    blueCars = car_cascade.detectMultiScale(blue, scaleValue, minNeigh)
    blackCars = car_cascade.detectMultiScale(black, scaleValue, minNeigh)
    whiteCars = car_cascade.detectMultiScale(white, scaleValue, minNeigh)

    # get the respective car color's bounding box for overlay
    drawCarBoundingBox(blueCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    drawCarBoundingBox(blackCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    drawCarBoundingBox(whiteCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

error: ignored

# **Challenge2 Functions**

In [ ]:
def drawWheelBoundingBox(carType, bbox_array, yCoordList, yInterval, boundaryBoxThickness):
  for (x,y,w,h) in carType: #x, y, width (max: 1280px), height (max: 720px)
    x += int(w*0.635)
    y += int(h*0.62)
    w = int(w*0.2)
    h = int(h*0.5)

    # get the average of the coordinate values
    yAvg = sum(yCoordList)
    if(len(yCoordList) > 0):
      yAvg /= len(yCoordList)
    # to prevent any non-vehicle areas to be mistaken as a vehicle
    if(len(yCoordList) <= 20) or (withinRange(y,yAvg,yInterval)): # the vehicle will only move on a x-axis basis, so only check y-axis change
      if(len(yCoordList) >= 20):
        yCoordList.pop(19) # remove last index of the coordList
      yCoordList.append(y) # add new coord to the list
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x + w, y + h),(132,245,66),boundaryBoxThickness)

# **Main Challenge2 & Challenge3** (wheel boundary)

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
boundaryBoxThickness = 5
scaleValue = 1.2
minNeigh = 3
yInterval = 150

# initialize the Haar Cascade car detection model
car_cascade = cv2.CascadeClassifier("CarSideViewXML.xml")
# initate a coordList that keeps the memory of the 20 y coordinates
yCoordList =[]

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"]) # convert JS response to OpenCV Image
    bbox_array = np.zeros([480,640,4], dtype = np.uint8) # create transparent overlay for bounding box

    # grayscale for the respective car color for car detection
    blue = detectColor(img, "blue") #cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    black = detectColor(img, "black")
    white = detectColor(img, "white")

    # get the region coordinates respective car color
    blueCars = car_cascade.detectMultiScale(blue, scaleValue, minNeigh)
    blackCars = car_cascade.detectMultiScale(black, scaleValue, minNeigh)
    whiteCars = car_cascade.detectMultiScale(white, scaleValue, minNeigh)

    # get the respective car color's front wheel bounding box for overlay
    drawWheelBoundingBox(blueCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    drawWheelBoundingBox(blackCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    drawWheelBoundingBox(whiteCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

NameError: ignored

# **Challenge4 Functions**

In [ ]:
def screenshot(filename, quality, img, x1, y1, x2, y2, boundaryBoxThickness):
  js = Javascript('''
    async function takePhoto(quality)
    {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')

  cv2.rectangle(img, (x1, y1),(x2, y2), (132,245,66), boundaryBoxThickness)
  cv2.imwrite(filename, img)
  return filename

In [ ]:
def drawWheelBoundingBoxV2(carType, bbox_array, yCoordList, yInterval, boundaryBoxThickness):
  xWheel = 0
  x1 = 0
  y1 = 0
  x2 = 0
  y2 = 0
  for (x,y,w,h) in carType: #x, y, width (max: 1280px), height (max: 720px)
    x += int(w*0.635)
    y += int(h*0.64)
    w = int(w*0.2)
    h = int(h*0.5)

    # get the average of the coordinate values
    yAvg = sum(yCoordList)
    if(len(yCoordList) > 0):
      yAvg /= len(yCoordList)
    # to prevent any non-vehicle areas to be mistaken as a vehicle
    if((len(yCoordList) <= 20) or (withinRange(y,yAvg,yInterval))): # the vehicle will only move on a x-axis basis, so only check y-axis change
      if(len(yCoordList) >= 20):
        yCoordList.pop(19) # remove last index of the coordList
      yCoordList.append(y) # add new coord to the list
      xWheel = (x - int((w/2)*0.635))
      x1 = x
      y1 = y
      x2 = x + w
      y2 = y + h
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x + w, y + h),(132,245,66),boundaryBoxThickness)
  return xWheel, x1, y1, x2, y2

# **Main Challenge4**

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
boundaryBoxThickness = 5
scaleValue = 1.2
minNeigh = 3
yInterval = 100

# initialize the Haar Cascade car detection model
car_cascade = cv2.CascadeClassifier("CarSideViewXML.xml")
# initate a coordList that keeps the memory of the 20 y coordinates
yCoordList =[]

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"]) # convert JS response to OpenCV Image
    bbox_array = np.zeros([480,640,4], dtype = np.uint8) # create transparent overlay for bounding box

    # grayscale for the respective car color for car detection
    blue = detectColor(img, "blue") #cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    black = detectColor(img, "black")
    white = detectColor(img, "white")

    # get the region coordinates respective car color
    blueCars = car_cascade.detectMultiScale(blue, scaleValue, minNeigh)
    blackCars = car_cascade.detectMultiScale(black, scaleValue, minNeigh)
    whiteCars = car_cascade.detectMultiScale(white, scaleValue, minNeigh)

    # get the respective car color's front wheel bounding box for overlay
    xWheel, x1, y1, x2, y2 = drawWheelBoundingBoxV2(blueCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    xWheel, x1, y1, x2, y2 = drawWheelBoundingBoxV2(blackCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    xWheel, x1, y1, x2, y2 = drawWheelBoundingBoxV2(whiteCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    # check if the xWheel value has reached the aluminium bar
    if(xWheel >= 640*0.53):
      filename = screenshot("Challenge4.jpg", 0.8, img, x1, y1, x2, y2, boundaryBoxThickness)
      print("Saved to Challenge4.jpg!")
      break

<IPython.core.display.Javascript object>

error: ignored

# **Distance Function**

In [ ]:
def dist_to_front_wheel(xWheel):
  dist = round((xWheel - 85) / (8.7/2), 3)
  return dist

# **Main Challenge5**

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
boundaryBoxThickness = 5
scaleValue = 1.2
minNeigh = 3
yInterval = 15

# initialize the Haar Cascade car detection model
car_cascade = cv2.CascadeClassifier("CarSideViewXML.xml")
# initate a coordList that keeps the memory of the 20 y coordinates
yCoordList =[]

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"]) # convert JS response to OpenCV Image
    bbox_array = np.zeros([480,640,4], dtype = np.uint8) # create transparent overlay for bounding box

    # grayscale for the respective car color for car detection
    blue = detectColor(img, "blue") #cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    black = detectColor(img, "black")
    white = detectColor(img, "white")

    # get the region coordinates respective car color
    blueCars = car_cascade.detectMultiScale(blue, scaleValue, minNeigh)
    blackCars = car_cascade.detectMultiScale(black, scaleValue, minNeigh)
    whiteCars = car_cascade.detectMultiScale(white, scaleValue, minNeigh)

    # get the respective car color's front wheel bounding box for overlay
    xWheel, x1, y1, x2, y2 = drawWheelBoundingBoxV2(blueCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    xWheel, x1, y1, x2, y2 = drawWheelBoundingBoxV2(blackCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)
    xWheel, x1, y1, x2, y2 = drawWheelBoundingBoxV2(whiteCars, bbox_array, yCoordList, yInterval, boundaryBoxThickness)

    # print out the cm on the video
    if((x2 > 0) or (y2 > 0)):
      distString = str(dist_to_front_wheel(xWheel)) + " cm"
      cv2.putText(bbox_array,distString,(x1,y1),3,1,(132,245,66),3)

    bbox_array = cv2.rectangle(bbox_array,(95,0),(96, 720),(132,245,66),2)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

error: ignored